# Show how to optimize the structure loaded in step 01
# Step 02 — Prepare grip regions

In [11]:
from pathlib import Path
import json
import numpy as np
from ase.io import read

import auto_prep

ROOT = Path.cwd()
WORKDIR = ROOT / "cases" / "case_center"

init_vasp = WORKDIR / "init.vasp"
assert init_vasp.exists(), f"Missing {init_vasp}"

atoms = read(str(init_vasp))
print("N_atoms =", len(atoms))
print("WORKDIR =", WORKDIR)

N_atoms = 359
WORKDIR = /Users/dawson666/Desktop/nano_tensile_TFvW/cases/case_center


## define the calculation

In [12]:
end_frac = 0.08
eps = 1e-3
min_layers = 2
max_layers = None

bottom_idx, top_idx = auto_prep.get_grip_indices(
    atoms,
    end_frac=end_frac,
    eps=eps,
    min_layers=min_layers,
    max_layers=max_layers,
    debug=True,
)

print("Bottom atoms =", len(bottom_idx))
print("Top atoms    =", len(top_idx))

[AutoPrep] Atomic Z-range: 39.750566 A
           z_min=0.000000, z_max=39.750566
           End-fraction (per side): 0.0800  -> target_thickness_raw=3.180045 A
           thickness clip: [2.000, 8.000] -> target_thickness=3.180045 A
           eps=0.001, distinct_z_levels=18
           min_layers=2, max_layers=unlimited
[AutoPrep] Bottom: layers=2, atoms=41, z=[0.000000, 2.338269], thickness~=2.338000 A
[AutoPrep] Top:    layers=2, atoms=38, z=[37.412297, 39.750566], thickness~=2.339000 A
[AutoPrep] Gap (top_min - bottom_max): 35.074029 A
[AutoPrep] Total grip fraction: 0.220056
Bottom atoms = 41
Top atoms    = 38


## configure the DFTpy run

In [13]:
np.save(WORKDIR / "bottom_idx.npy", bottom_idx)
np.save(WORKDIR / "top_idx.npy", top_idx)

assert len(set(bottom_idx) & set(top_idx)) == 0
assert len(bottom_idx) > 0 and len(top_idx) > 0

grip_meta = {
    "end_frac": float(end_frac),
    "eps": float(eps),
    "min_layers": int(min_layers),
    "max_layers": None if max_layers is None else int(max_layers),
    "n_bottom": int(len(bottom_idx)),
    "n_top": int(len(top_idx)),
}
(WORKDIR / "grip_meta.json").write_text(json.dumps(grip_meta, indent=2))

print("Saved:", WORKDIR / "bottom_idx.npy")
print("Saved:", WORKDIR / "top_idx.npy")
print("Saved:", WORKDIR / "grip_meta.json")
print("Grip regions OK")

Saved: /Users/dawson666/Desktop/nano_tensile_TFvW/cases/case_center/bottom_idx.npy
Saved: /Users/dawson666/Desktop/nano_tensile_TFvW/cases/case_center/top_idx.npy
Saved: /Users/dawson666/Desktop/nano_tensile_TFvW/cases/case_center/grip_meta.json
Grip regions OK


In [ ]:
from ase.io import read, write
import main
import dft_engine

pp = ROOT / "al.gga.psp"
assert pp.exists(), f"Missing {pp}"

ecut_ev = 500.0
spacing = float(main.ecut_to_spacing_angstrom(ecut_ev))

bottom_idx = np.load(WORKDIR / "bottom_idx.npy")
top_idx = np.load(WORKDIR / "top_idx.npy")
fixed_idx = np.unique(np.r_[bottom_idx, top_idx]).astype(int)

atoms0 = read(str(WORKDIR / "init.vasp"))

atoms0_rlx, E0, S0 = dft_engine.relax_atoms(
    atoms0,
    pp_file=pp,
    spacing=spacing,
    fixed_idx=fixed_idx,
    fmax=0.05,
    steps=200,
    logfile=str(WORKDIR / "cycle_000_relax.log"),
    trajfile=str(WORKDIR / "cycle_000_relax.traj"),
    dftpy_outfile=str(WORKDIR / "cycle_000_dftpy.out"),
    debug_fixed=True,
)

ref_xyz = WORKDIR / "cycle_000_relaxed.xyz"
write(str(ref_xyz), atoms0_rlx)

ref = {"E_ref_eV": float(E0), "ecut_ev": float(ecut_ev), "spacing_A": float(spacing)}
ref_json = WORKDIR / "ref_energy.json"
ref_json.write_text(json.dumps(ref, indent=2))

print("E_ref_eV =", f"{E0:.12f}")
print("Saved:", ref_xyz)
print("Saved:", ref_json)

Saved:
 - /Users/dawson666/Desktop/nano_tensile_TFvW/cases/case_center/bottom_idx.npy
 - /Users/dawson666/Desktop/nano_tensile_TFvW/cases/case_center/top_idx.npy


In [15]:
from pathlib import Path
import json
import numpy as np
from ase.io import read, write
import main
import dft_engine

ROOT = Path.cwd()
WORKDIR = ROOT / "cases" / "case_center"
pp = ROOT / "al.gga.psp"

assert (WORKDIR / "init.vasp").exists()
assert (WORKDIR / "bottom_idx.npy").exists()
assert (WORKDIR / "top_idx.npy").exists()
assert pp.exists()

ecut_ev = 500.0
spacing = float(main.ecut_to_spacing_angstrom(ecut_ev))

bottom_idx = np.load(WORKDIR / "bottom_idx.npy")
top_idx = np.load(WORKDIR / "top_idx.npy")
fixed_idx = np.unique(np.r_[bottom_idx, top_idx]).astype(int)

atoms0 = read(str(WORKDIR / "init.vasp"))

atoms0_rlx, E0, S0 = dft_engine.relax_atoms(
    atoms0,
    pp_file=pp,
    spacing=spacing,
    fixed_idx=fixed_idx,
    fmax=0.05,
    steps=200,
    logfile=str(WORKDIR / "cycle_000_relax.log"),
    trajfile=str(WORKDIR / "cycle_000_relax.traj"),
    dftpy_outfile=str(WORKDIR / "cycle_000_dftpy.out"),
    debug_fixed=True,
)

ref_xyz = WORKDIR / "cycle_000_relaxed.xyz"
write(str(ref_xyz), atoms0_rlx)

ref_json = WORKDIR / "ref_energy.json"
ref_json.write_text(json.dumps(
    {"E_ref_eV": float(E0), "ecut_ev": float(ecut_ev), "spacing_A": float(spacing)},
    indent=2
))

print("Saved:", ref_xyz, "exists:", ref_xyz.exists())
print("Saved:", ref_json, "exists:", ref_json.exists())
print("E_ref_eV =", f"{E0:.12f}")

[relax_atoms] before run fixed_z(min/max)=0.000000/39.750566
The final grid size is  [ 78  78 154]
setting key: Al -> /Users/dawson666/Desktop/nano_tensile_TFvW/al.gga.psp
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       2.792476049202E+03      2.792476E+03    2.840297E+04    1       1       3.103790E-01    
!WARN : pAp small than zero :iter =  1 -3993145.5123826605
1       -4.954174877307E+02     -3.287894E+03   1.043612E+03    2       2       7.113459E-01    
2       -7.047429774287E+02     -2.093255E+02   1.807121E+02    3       1       1.098303E+00    
!WARN : pAp small than zero :iter =  7 -28075.660393028054
3       -7.304799475177E+02     -2.573697E+01   5.121225E+01    8       2       2.039878E+00    
4       -7.493893270876E+02     -1.890938E+01   5.140470E+00    12      2       3.291180E+00    
5       -7.502926091373E+02     -9.032820E-01   6.743429E-01    10      2       4.367210E+00    
6       -7.504646827237E+02    

In [16]:
ref_xyz = WORKDIR / "cycle_000_relaxed.xyz"
assert ref_xyz.exists(), "Reference relaxed structure not written"

ref_json = WORKDIR / "ref_energy.json"
assert ref_json.exists(), "Reference energy json not written"

print("Reference relax OK")

Reference relax OK
